In [1]:
import sys
sys.path.append('../')
import pnn
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [21]:
class original_model_template(torch.nn.Module):
  def __init__(self, embed_module, interact_modules, agg_module):
    super().__init__()
    self.embed_module = embed_module
    self.interact_modules = nn.ModuleList(interact_modules)
    self.agg_module = agg_module
    
  def forward(self, x):
    embed = self.embed_module(x)
    interact_results = [self.interact_modules[i](embed) for i in range(len(self.interact_modules))]
    return self.agg_module(interact_results)

In [ ]:
class fused_model_template(torch.nn.Module):
  def __init__(self, embed_module, interact_modules, fuse_module, modify_embedding_module, interact_modules_input_type):
    super().__init__()
    self.embed_module = embed_module
    self.interact_modules = nn.ModuleList(interact_modules)
    self.fuse_module = fuse_module
    self.interact_modules_input_type = interact_modules_input_type
    self.modify_embedding_module = modify_embedding_module
  
  def process(self, i, embed, x):
    if self.interact_modules_input_type[i] == "embed":
        return self.interact_modules[i](embed)
    else:
        return self.interact_modules[i](x)    
      
  def forward(self, x):
    if self.modify_embedding_module:
      embed = self.embed_module(x)
      interact_results = [self.process(i, embed, x)  for i in range(len(self.interact_modules))]
    else:
      embed = x 
      interact_results = [self.interact_modules[i](embed)  for i in range(len(self.interact_modules))]
    return self.fuse_module(interact_results)

In [22]:
class embed_module_single(torch.nn.Module):
  def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)  
      
  def forward(self, x):
    return self.embed(x)

In [53]:
class product_sum_module(torch.nn.Module):
  def __init__(self):
      super().__init__()
  
  def forward(self, x):
    square_of_sum = torch.sum(x, dim=1) ** 2
    sum_of_square = torch.sum(x ** 2, dim=1)
    ix = square_of_sum - sum_of_square
    ix = torch.sum(ix, dim=1, keepdim=True)
    return 0.5 * ix

In [42]:
class mlp_module(torch.nn.Module):
  def __init__(self, num_fields, embed_dim):
    super().__init__()
    self.mlp_layer = torch.nn.Linear(1, 1)
    self.len_of_embed = num_fields * embed_dim
    
  def forward(self, x):
    return self.mlp_layer(x.view(-1,self.len_of_embed))

In [80]:
class no_agg_module(torch.nn.Module):
  def __init__(self):
    super().__init__()  
  
  def forward(self, x):
    return x

In [93]:
t = original_model_template(embed_module_single(), [product_sum_module(),mlp_module(100,32) ],no_agg_module())

In [94]:
traced = symbolic_trace(t)

In [95]:
traced.print_readable()

class original_model_template(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        embed_module_embed = self.embed_module.embed(x);  x = None
        sum_1 = torch.sum(embed_module_embed, dim = 1)
        pow_1 = sum_1 ** 2;  sum_1 = None
        pow_2 = embed_module_embed ** 2
        sum_2 = torch.sum(pow_2, dim = 1);  pow_2 = None
        sub = pow_1 - sum_2;  pow_1 = sum_2 = None
        sum_3 = torch.sum(sub, dim = 1, keepdim = True);  sub = None
        mul = 0.5 * sum_3;  sum_3 = None
        view = embed_module_embed.view(-1, 3200);  embed_module_embed = None
        interact_modules_1_mlp_layer = getattr(self.interact_modules, "1").mlp_layer(view);  view = None
        return [mul, interact_modules_1_mlp_layer]
        


'class original_model_template(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        embed_module_embed = self.embed_module.embed(x);  x = None\n        sum_1 = torch.sum(embed_module_embed, dim = 1)\n        pow_1 = sum_1 ** 2;  sum_1 = None\n        pow_2 = embed_module_embed ** 2\n        sum_2 = torch.sum(pow_2, dim = 1);  pow_2 = None\n        sub = pow_1 - sum_2;  pow_1 = sum_2 = None\n        sum_3 = torch.sum(sub, dim = 1, keepdim = True);  sub = None\n        mul = 0.5 * sum_3;  sum_3 = None\n        view = embed_module_embed.view(-1, 3200);  embed_module_embed = None\n        interact_modules_1_mlp_layer = getattr(self.interact_modules, "1").mlp_layer(view);  view = None\n        return [mul, interact_modules_1_mlp_layer]\n        '

In [66]:
traced.graph.print_tabular()

opcode         name                          target                                                  args                                    kwargs
-------------  ----------------------------  ------------------------------------------------------  --------------------------------------  ---------------------------
placeholder    x                             x                                                       ()                                      {}
call_module    embed_module_embed            embed_module.embed                                      (x,)                                    {}
call_function  sum_1                         <built-in method sum of type object at 0x7f5f2be81760>  (embed_module_embed,)                   {'dim': 1}
call_function  pow_1                         <built-in function pow>                                 (sum_1, 2)                              {}
call_function  pow_2                         <built-in function pow>                               

In [32]:
import dfm

In [86]:
dfm_model = dfm.DeepFactorizationMachineModel([100 for _ in range(100)],32,[400,400,400],0.1)

In [87]:
dfm_model_traced = symbolic_trace(dfm_model)

In [88]:
dfm_model_traced.print_readable()

class DeepFactorizationMachineModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        embedding_offsets = self.embedding.offsets
        add = x + embedding_offsets;  embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add);  add = None
        linear_offsets = self.linear.offsets
        add_1 = x + linear_offsets;  x = linear_offsets = None
        linear_fc = self.linear.fc(add_1);  add_1 = None
        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None
        linear_bias = self.linear.bias
        add_2 = sum_1 + linear_bias;  sum_1 = linear_bias = None
        sum_2 = torch.sum(embedding_embedding, dim = 1)
        pow_1 = sum_2 ** 2;  sum_2 = None
        pow_2 = embedding_embedding ** 2
        sum_3 = torch.sum(pow_2, dim = 1);  pow_2 = None
        sub = pow_1 - sum_3;  pow_1 = sum_3 = None
        sum_4 = torch.sum(sub, dim = 1, keepdim = True);  sub = None
        mul = 0.5 * sum_4;  sum_4

'class DeepFactorizationMachineModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        embedding_offsets = self.embedding.offsets\n        add = x + embedding_offsets;  embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add);  add = None\n        linear_offsets = self.linear.offsets\n        add_1 = x + linear_offsets;  x = linear_offsets = None\n        linear_fc = self.linear.fc(add_1);  add_1 = None\n        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None\n        linear_bias = self.linear.bias\n        add_2 = sum_1 + linear_bias;  sum_1 = linear_bias = None\n        sum_2 = torch.sum(embedding_embedding, dim = 1)\n        pow_1 = sum_2 ** 2;  sum_2 = None\n        pow_2 = embedding_embedding ** 2\n        sum_3 = torch.sum(pow_2, dim = 1);  pow_2 = None\n        sub = pow_1 - sum_3;  pow_1 = sum_3 = None\n        sum_4 = torch.sum(sub, dim = 1, keepdim = True);  sub = None\n        mul = 

In [89]:
dfm_model_traced.graph.print_tabular()

opcode         name                 target                                                      args                             kwargs
-------------  -------------------  ----------------------------------------------------------  -------------------------------  ---------------------------
placeholder    x                    x                                                           ()                               {}
get_attr       embedding_offsets    embedding.offsets                                           ()                               {}
call_function  add                  <built-in function add>                                     (x, embedding_offsets)           {}
call_module    embedding_embedding  embedding.embedding                                         (add,)                           {}
get_attr       linear_offsets       linear.offsets                                              ()                               {}
call_function  add_1                <built-in f

In [96]:
matcher =  SubgraphMatcher(traced.graph, match_output=False, match_placeholder=False,
                            remove_overlapping_matches=True)

In [97]:
_matches = matcher.match(dfm_model_traced.graph)

In [98]:
_matches

[InternalMatch(anchors=[mul, interact_modules_1_mlp_layer], nodes_map={mul: mul, sum_3: sum_4, sub: sub, pow_1: pow_1, sum_1: sum_2, embed_module_embed: embedding_embedding, x: add, sum_2: sum_3, pow_2: pow_2, interact_modules_1_mlp_layer: mlp_mlp_0, view: view}, placeholder_nodes=[add], returning_nodes=[mul, mlp_mlp_0], name_node_map={})]